# 피드백 루프를 포함한 RAG 시스템: 검색 및 응답 품질 향상

## 개요

이 시스템은 피드백 루프가 통합된 Retrieval-Augmented Generation (RAG) 접근 방식을 구현하여, 사용자 피드백을 활용해 검색 과정을 동적으로 조정하고, 시간이 지남에 따라 응답의 품질과 관련성을 향상시키는 것을 목표로 합니다.

## 동기

기존의 RAG 시스템은 검색 과정의 한계나 지식 베이스의 제약으로 인해 일관성 없는 응답이나 관련성이 낮은 결과를 생성할 수 있습니다. 피드백 루프를 구현함으로써 다음과 같은 이점을 얻을 수 있습니다:

1. 검색된 문서의 품질을 지속적으로 개선
2. 생성된 응답의 관련성을 강화
3. 시간에 따라 사용자 선호도와 요구 사항에 맞추어 시스템이 적응하도록 지원

## 방법 설명

### 1. 초기 설정
- 시스템이 PDF 콘텐츠를 읽어 벡터 스토어를 생성
- 벡터 스토어를 기반으로 검색기 초기화
- 응답 생성을 위해 언어 모델(LLM) 설정

### 2. 쿼리 처리
- 사용자가 쿼리를 제출하면 검색기가 관련 문서를 검색
- 검색된 문서를 바탕으로 LLM이 응답 생성

### 3. 피드백 수집
- 응답의 관련성과 품질에 대한 사용자 피드백을 수집
- 피드백은 JSON 파일에 영구 저장

### 4. 관련성 점수 조정
- 이후 쿼리에서는 이전 피드백을 로드하여 활용
- LLM이 현재 쿼리에 대해 과거 피드백의 관련성을 평가
- 이 평가를 바탕으로 문서의 관련성 점수를 조정

### 5. 검색기 업데이트
- 조정된 문서 점수로 검색기를 업데이트하여 향후 검색에 피드백 반영

### 6. 주기적 인덱스 미세 조정
- 일정 주기로 시스템이 인덱스를 미세 조정
- 고품질 피드백을 사용해 추가 문서를 생성
- 새로운 문서로 벡터 스토어를 업데이트하여 전체 검색 품질을 향상

## 이 접근 방식의 이점

1. **지속적 개선**: 시스템이 각 상호작용을 통해 점차 성능을 향상시킴
2. **개인화**: 사용자 피드백을 반영하여 시간에 따라 개인이나 그룹의 선호도에 맞게 적응
3. **관련성 증가**: 피드백 루프가 미래 검색에서 더 관련성 높은 문서를 우선시하도록 도움
4. **품질 관리**: 시간이 지남에 따라 저품질 또는 관련성이 낮은 응답이 줄어듦
5. **적응성**: 사용자 요구나 문서 내용의 변화에 따라 시스템이 유연하게 조정

## 결론

이 피드백 루프가 포함된 RAG 시스템은 기존 RAG 시스템에 비해 큰 발전을 이룹니다. 사용자 상호작용에서 지속적으로 학습함으로써 보다 동적이고, 적응적이며, 사용자 중심의 정보 검색 및 응답 생성 방식을 제공합니다. 정보 정확성과 관련성이 중요한 분야에서 특히 가치가 크며, 사용자 요구가 시간이 지남에 따라 발전하는 경우 유용한 시스템입니다.

이 구현은 기본 RAG 시스템에 비해 복잡성을 추가하지만, 응답 품질과 사용자 만족도 측면에서 큰 이점을 제공하므로 높은 품질의 컨텍스트 인식 정보 검색 및 생성이 필요한 애플리케이션에서 투자 가치가 충분합니다.


<div style="text-align: center;">

<img src="../images/retrieval_with_feedback_loop.svg" alt="retrieval with feedback loop" style="width:40%; height:auto;">
</div>

### Import relevant libraries

In [34]:
import os
import sys
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
import json
from typing import List, Dict, Any


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### Define documents path

In [35]:
path = "../data/Understanding_Climate_Change.pdf"

### Create vector store and retrieval QA chain

In [36]:
content = read_pdf_to_string(path) # pdf 문자열로 변환 
vectorstore = encode_from_string(content) # 내용을 인코딩하여 벡터저장소
retriever = vectorstore.as_retriever() # 벡터저장소 기반으로 하여 리트리버 

llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever) # gpt4를 활용하여 리트리브하는 체인 

In [37]:
content

'Understanding Climate Change \nChapter 1: Introduction to Climate Change \nClimate change refers to significant, long-term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past century, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change. \nHistorical Context \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which \nbegan at the end of the last ice a

### Function to format user feedback in a dictionary

In [38]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments
    }

### 피드백을 저장하는 함수 

In [39]:
def store_feedback(feedback):
    with open("../data/feedback_data.json", "a") as f:
        json.dump(feedback, f)
        f.write("\n")

### 피드백 파일 읽는 함수

In [40]:
def load_feedback_data():
    feedback_data = []
    try:
        with open("../data/feedback_data.json", "r") as f:
            for line in f:
                feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    return feedback_data

### Function to adjust files relevancy based on the feedbacks file
### 문서와 피드백이 관련성이 있을 때 피드백을 리스트에 저장함 -> 

In [41]:
from typing import List, Any, Dict
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# 응답 모델 정의
class Response(BaseModel):
    # 답변이 'Yes' 또는 'No'로만 제한된 필드
    answer: str = Field(..., title="The answer to the question. The options can be only 'Yes' or 'No'")
# qeury는 현재 쿼리
# feedback qeury는 과거 피드백 받은 내용에 대한 쿼리 
def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]]) -> List[Any]:
    # 관련성 확인을 위한 프롬프트 템플릿 생성
    relevance_prompt = PromptTemplate(
        input_variables=["query", "feedback_query", "doc_content", "feedback_response"],
        template="""
        Determine if the following feedback response is relevant to the current query and document content.
        You are also provided with the Feedback original query that was used to generate the feedback response.
        Current query: {query} 
        Feedback query: {feedback_query}
        Document content: {doc_content}
        Feedback response: {feedback_response}
        
        Is this feedback relevant? Respond with only 'Yes' or 'No'.
        """
    )
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

    # LLMChain을 사용하여 프롬프트와 LLM을 연결
    relevance_chain = LLMChain(prompt=relevance_prompt, llm=llm)

    for doc in docs:
        relevant_feedback = []
        
        for feedback in feedback_data:
            # LLM을 사용하여 입력 데이터로 관련성 확인
            input_data = {
                "query": query,
                "feedback_query": feedback['query'],
                "doc_content": doc.page_content[:1000],
                "feedback_response": feedback['response']
            }
            # 관련성 평가 결과 받기
            result = relevance_chain.run(input_data).strip()
            
            # 결과가 'Yes'인 경우 관련 피드백 리스트에 추가
            if result.lower() == 'yes':
                relevant_feedback.append(feedback)
        
        # 관련 피드백이 있는 경우 문서의 관련성 점수 조정  -> 사용자가 긍정적인 피드백을 줬을 경우 나중에 비슷한 질문을 했을 때 더 쉽게 반환할 수 있도록 하려고 
        if relevant_feedback:
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            doc.metadata['relevance_score'] *= (avg_relevance / 3)  # 1-5 점수 척도에서 중립을 3으로 가정
    
    # 조정된 관련성 점수를 기준으로 문서를 내림차순으로 정렬하여 반환
    return sorted(docs, key=lambda x: x.metadata['relevance_score'], reverse=True)


### 기존 텍스트와 고품질 추가 텍스트를 결합하여 새로운 벡터 스토어 생성 
- 쿼리 + 질 좋은 문서 로 만들어버림
- 그럼으로써 나중에 유사한 질문을 했을 때 바로 답변할 수 있도록 

In [51]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], texts: List[str]) -> Any:
    # 높은 피드백을 저장 
    good_responses = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]
    
    # 높은 피드백들을 각각 쿼리와 합쳐서 새롭게 텍스트 리스트를 만듬 
    additional_texts = []
    for f in good_responses:
        combined_text = f['query'] + " " + f['response']
        additional_texts.append(combined_text)

    # 리스트를 이어버림 
    additional_texts = " ".join(additional_texts)
    
    # Create a new index with original and high-quality texts
    all_texts = texts + additional_texts
    new_vectorstore = encode_from_string(all_texts)
    
    return new_vectorstore , all_texts, additional_texts 

### Demonstration of how to retrieve answers with respect to user feedbacks

In [43]:
# 1. llm으로 질문에 대한 대답을 함 
query = "What is the greenhouse effect?"

# 2. 질문에 대한 대답을 피드백으로 넣음 
# Get response from RAG system
response = qa_chain(query)["result"]
response

"The greenhouse effect is a natural process where greenhouse gases in the Earth's atmosphere trap heat from the sun. These gases, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), allow sunlight to enter the atmosphere and reach the Earth's surface. When the Earth's surface absorbs this sunlight, it emits heat in the form of infrared radiation. Greenhouse gases absorb and re-radiate some of this heat, preventing it from escaping back into space, thus warming the planet. This effect is essential for life on Earth, as it keeps the planet warm enough to support life. However, human activities have intensified this natural process, leading to a warmer climate."

In [44]:
# 3 사용자가 응답에 대해 높은 피드백점수를 줌 
relevance = 5
quality = 5

feedback = get_user_feedback(query, response, relevance, quality)
feedback
# 이를 저장 
store_feedback(feedback)

In [57]:
load_feedback_data()
feedback

{'query': 'What is the greenhouse effect?',
 'response': "The greenhouse effect is a natural process where greenhouse gases in the Earth's atmosphere trap heat from the sun. These gases, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), allow sunlight to enter the atmosphere and reach the Earth's surface. When the Earth's surface absorbs this sunlight, it emits heat in the form of infrared radiation. Greenhouse gases absorb and re-radiate some of this heat, preventing it from escaping back into space, thus warming the planet. This effect is essential for life on Earth, as it keeps the planet warm enough to support life. However, human activities have intensified this natural process, leading to a warmer climate.",
 'relevance': 5,
 'quality': 5,
 'comments': ''}

In [46]:
# 쿼리 통해 관련된 문서를 뽑아냄 
docs = retriever.get_relevant_documents(query)

# 관련문서랑, 쿼리, 피드백을 함수에 넣어 피드백에 대한 점수를 매김 
adjusted_docs = adjust_relevance_scores(query, docs, load_feedback_data())


In [47]:
adjusted_docs

[Document(metadata={'relevance_score': 1.6666666666666667}, page_content='The use of synthetic fertilizers in agriculture releases nitrous oxide, a potent greenhouse gas. \nPractices such as precision farming and organic fertilizers can mitigate these emissions. The \ndevelopment of eco-friendly fertilizers and farming techniques is essential for reducing the \nagricultural sector\'s carbon footprint. \nChapter 3: Effects of Climate Change \nThe effects of climate change are already being felt around the world and are projected to \nintensify in the coming decades. These effects include: \nRising Temperatures \nGlobal temperatures have risen by about 1.2 degrees Celsius (2.2 degrees Fahrenheit) since \nthe late 19th century. This warming is not uniform, with some regions experiencing more \nsignificant increases than others. \nHeatwaves \nHeatwaves are becoming more frequent and severe, posing risks to human health, agriculture, \nand infrastructure. Cities are particularly vulnerable 

In [48]:

# retreiver 업데이트 
retriever.search_kwargs['k'] = len(adjusted_docs)
retriever.search_kwargs['docs'] = adjusted_docs

In [50]:
content

'Understanding Climate Change \nChapter 1: Introduction to Climate Change \nClimate change refers to significant, long-term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past century, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change. \nHistorical Context \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which \nbegan at the end of the last ice a

### Finetune the vectorstore periodicly

In [53]:
# Periodically (e.g., daily or weekly), fine-tune the index
new_vectorstore = fine_tune_index(load_feedback_data(), content)
retriever = new_vectorstore[0].as_retriever()

In [54]:
new_vectorstore
# 첫번째는 벡터스토어
# 두번째는 기존 텍스트 + additional_texts
# 세번째는 additional_texts -> 질문과 피드백 좋게 받은 답변 

(<langchain_community.vectorstores.faiss.FAISS at 0x17bc389b0>,
 'Understanding Climate Change \nChapter 1: Introduction to Climate Change \nClimate change refers to significant, long-term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past century, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change. \nHistorical Context \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. Duri

# RAG 시스템을 개선하는 피드백 기반 검색 품질 향상 코드

이 코드는 **사용자 피드백을 활용하여 RAG 시스템의 검색 품질을 점진적으로 개선**하는 작업을 수행합니다.

## 코드 흐름

1. **쿼리 및 응답 생성**  
   - 사용자가 쿼리를 입력하면, RAG 시스템이 적합한 응답을 생성합니다.

2. **피드백 수집 및 저장**
   - 사용자는 응답의 관련성(`relevance`)과 품질(`quality`)을 평가하여 피드백을 제공합니다.
   - 피드백은 향후 검색에 반영될 수 있도록 데이터베이스에 저장됩니다.

3. **관련성 점수 조정**
   - 저장된 피드백을 기반으로, `adjust_relevance_scores` 함수가 문서의 관련성 점수를 조정합니다.
   - 이를 통해 **미래의 유사한 쿼리에서 더 관련성 높은 문서가 상위에 노출**될 수 있도록 합니다.
   - 조정된 점수를 기준으로 문서들이 재정렬됩니다.

4. **검색기 업데이트**
   - 조정된 문서 리스트(`adjusted_docs`)로 검색기(`retriever`)를 업데이트하여 **최신 피드백이 반영된 상태**로 유지합니다.

5. **주기적 인덱스 미세 조정**
   - `fine_tune_index` 함수는 고품질 피드백 데이터를 반영하여 새로운 벡터 스토어를 생성하고, 이를 검색 인덱스에 적용하여 주기적으로 검색 품질을 개선합니다.

## 요약
이 흐름을 통해 사용자의 피드백이 반영된 최신 검색 인덱스를 유지하여, RAG 시스템이 **더욱 정확하고 관련성 높은 검색 결과**를 제공할 수 있도록 합니다.


## 내 생각
- 질문을 했을 때 그 질문에 대해 사용자가 평점을 매김 - 관련성과 퀄리티, 코멘트도 남김 
- 나중에 어떤 질문을 했을 때 현재 질문, 과거 피드백 받은 질문, 그 질문에 대한 답변, 현질문에 관련있는 retrive된 문서 를 넣음 
- llm이 현재 질문과 질문에 관려된 문서가 (과거 질문과 과거질문에 대한 답변)이 얼마나 유사한지 평가한 후 유의미하다는 yes가 나오면 점수를 다시 매겨서 해당 점수 기준으로 문서 재정렬
- 조정된 문서 리스트로 검색기 업데이트 한다. 